In [1]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification, create_optimizer
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.keras.utils import pad_sequences
import datasets

https://huggingface.co/docs/transformers/tasks/sequence_classification

https://iq.opengenus.org/binary-text-classification-bert/

https://towardsdatascience.com/text-classification-with-bert-in-pytorch-887965e5820f

In [12]:
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [15]:
def read_file(file):
    df = pd.read_json(file)
    return df

### Leitura de arquivo

In [51]:
df = read_file("DIR_CON_ME_2000_2025_20240406_102248.json")
df = pd.concat([df.drop(['options'], axis=1), pd.json_normalize(df['options'])], axis=1)
df = df.iloc[:, :-1]
df.head()

,id,subject,themes,year,examiner,entity,enunciation,nullified,outdated,answer,A,B,C,D,E
0,Q2222376,Direito Constitucional,[Tribunais Regionais Eleitorais e Juízes Eleit...,2001,NCE-UFRJ,TRE-RJ,A correta composição do Tribunal Regional Elei...,False,False,C,"sete juízes, sendo um desembargador, três juíz...","nove juízes, sendo dois desembargadores, dois ...","sete juízes, sendo dois desembargadores, dois ...","nove juízes, sendo três desembargadores, dois ...","no mínimo sete juízes, sendo dois desembargado..."
1,Q2222375,Direito Constitucional,[Administração Pública – Disposições Gerais e ...,2001,NCE-UFRJ,TRE-RJ,A Constituição Federal assegura aos servidores...,False,False,B,"seguro-desemprego, fundo de garantia do tempo ...",remuneração do trabalho noturno superior à do ...,"aviso prévio proporcional ao tempo de serviço,...",proibição de trabalho noturno a menores de dez...,duração do trabalho normal não superior a oito...
2,Q2222374,Direito Constitucional,[Administração Pública – Disposições Gerais e ...,2001,NCE-UFRJ,TRE-RJ,Assinale a opção que contém apenas princípios ...,False,False,C,"legalidade, imparcialidade, moralidade, eficiê...","impessoalidade, imparcialidade, moralidade, pu...","publicidade, legalidade, moralidade, eficiênci...","inafastabilidade, publicidade, legalidade, efi...","moralidade ,inamovibilidade, legalidade, publi..."
3,Q2222373,Direito Constitucional,[Direitos Políticos],2001,NCE-UFRJ,TRE-RJ,Podem concorrer validamente ao cargo de Govern...,False,False,D,brasileiro naturalizado e Governador já reelei...,prefeito reeleito e mulher de Governador de Es...,analfabeto e Deputado Federal;,o Governador substituto de Governador falecido...,"brasileiro nato, com 21 anos de idade e Senador."
4,Q2222372,Direito Constitucional,[Direitos da Nacionalidade],2001,NCE-UFRJ,TRE-RJ,Um brasileiro naturalizado e um brasileiro nat...,False,False,A,Presidente da Câmara dos Deputados e Senador;,Deputado Federal e juiz de paz;,Senador e Governador de Estado;,Presidente do Senado Federal e Vereador;,Ministro do Supremo Tribunal Federal e Deputad...


In [52]:
print(f"Antes de dropar: {len(df)}")
df = df.dropna(subset=["A", "B", "C", "D"])
print(f"Depois de dropar: {len(df)}")

Antes de dropar: 38823
Depois de dropar: 38800


Cada coluna de resposta ficará em uma linha agora

In [55]:
df_exp = df.melt(id_vars=["id", "subject", "themes", "year", "examiner", "entity", "enunciation", "nullified", "outdated", "answer"], value_vars=["A", "B", "C", "D", "E"], var_name="candidate_answer_id", value_name='candidate_answer_text')
df_exp = df_exp.dropna(subset=["candidate_answer_text"])

In [63]:
def preprocess_function(df: pd.DataFrame):
    enunciations = df['enunciation'].to_list()
    answers = df["candidate_answer_text"].to_list()

    tokenized_examples = tokenizer(enunciations, answers, max_length = MAX_LENGTH, truncation=True)
    return {k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

def tokenize(df: pd.DataFrame, MAX_LENGTH = 512):
  def tokenize(sentence: str):
    return tokenizer(sentence, padding='max_length', max_length = MAX_LENGTH, truncation=True, return_tensors="tf")

  return [tokenize(sentence) for sentence in df['enunciation']]

In [59]:
x = [['m','oi'], ['m','oi2']]
sum(x, [])

['m', 'oi', 'm', 'oi2']

In [64]:
x = preprocess_function(df_exp)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


MemoryError: 

In [5]:
tokenized = tokenize(df)

NameError: name 'df' is not defined

In [ ]:
example_text = tokenizer.decode(tokenized[0]['input_ids'][0])
example_text

'[CLS] Julgue o item a seguir, relativo aos servidores públicos e à administração pública. Caso ficasse demonstrado, em sede de processo administrativo regularmente realizado, que um agente de polícia federal recebera R $ 20. 000, 00 para deixar de realizar a prisão em flagrante de um traficante de drogas, então haveria de ser - lhe imposta, administrativamente, a pena de demissão. Todavia, se o agente fosse processado criminalmente pela prática do mesmo fato, simultaneamente à tramitação do processo administrativo, ele só poderia ser demitido após o trânsito em julgado da respectiva sentença condenatória. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

### Classificação

In [ ]:
id2label = {0: "Errado", 1: "Certo"}
label2id = {"Certo": 0, "Errado": 1}

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=2, id2label=id2label, label2id=label2id)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
